# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# Importing libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
import pickle
import sys
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Define the path to the database file
db_path = "Data/DisasterResponse.db"

# Create a SQLAlchemy engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{db_path}')

# Retrieve data from the "WB_disaster_messages" table in the database
df = pd.read_sql_table("DisasterResponse", con=engine)

In [ ]:
df.head()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """
    Tokenize the input text.
    
    Args:
    text (str): The text to be tokenized.
    
    Returns:
    list: A list of cleaned and tokenized words.
    """
    # Normalize text
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stop words
    words = [word for word in tokens if word not in stopwords.words("english")]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word).strip() for word in words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# Convert category values to just numbers 0 or 1
for column in df.columns[4:]:
    df[column] = df[column].astype(str).str[-1]  # Take the last character and convert to string
    df[column] = df[column].astype(int)  # Convert to integer

# Define feature and target variables
X = df['message']
Y = df.drop(columns=['message', 'id', 'original', 'genre'])

# Create a machine learning pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Train the pipeline
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Predict on test data
Y_pred = pipeline.predict(X_test)

# Evaluate the model
for i, column in enumerate(Y.columns):
    print(f"Category: {column}")
    print(classification_report(Y_test[column], Y_pred[:, i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Define the parameter grid
parameters = {
    'tfidf__max_df': [0.75, 1.0],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, Y_train)

# Get best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Use the best parameters to predict on test data
best_model = grid_search.best_estimator_
Y_pred = best_model.predict(X_test)

# Evaluate the model
for i, column in enumerate(Y.columns):
    print(f"Category: {column}")
    print(classification_report(Y_test[column], Y_pred[:, i]))


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Use the best parameters to predict on test data
best_model = grid_search.best_estimator_
Y_pred = best_model.predict(X_test)

# Evaluate the model
overall_accuracy = (Y_pred == Y_test.values).mean().mean()
overall_precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=0)
overall_recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=0)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")

# Print classification report for each category
for i, column in enumerate(Y.columns):
    print(f"Category: {column}")
    print(classification_report(Y_test[column], Y_pred[:, i], zero_division=0))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# Assuming your trained model is named 'model'
# and you want to save it to a file named 'model.pkl'
filename = 'Modeling/model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.